In [44]:
import sys

sys.path.append(r'C:\users\17049\appdata\local\programs\python\python311\lib\site-packages')

import pandas as pd
import fredapi as fa
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Source: https://fred.stlouisfed.org/
fred = fa.Fred(api_key =os.environ.get('FRED_API_KEY'))

#Get UST10 Yield
ten = fred.get_series('DGS30')
ten.name = 'UST 30 Year Yield'
df = pd.DataFrame(ten)

#Get ave 30 yr mortgage data
dfm = fred.get_series('MORTGAGE30US')
dfm.name = 'Ave US 30 Year Mortgage Rate'
dfm = pd.DataFrame(dfm)

#Get Origination data
o = fred.get_series('RCMFLOORIG')
o.name = 'Qtrly Originations (B)'
dfo = pd.DataFrame(o)

m = pd.merge(df, dfm, left_index=True, right_index=True).dropna()

m['Basis'] = m['Ave US 30 Year Mortgage Rate'] - m['UST 30 Year Yield']

m.head()

,UST 30 Year Yield,Ave US 30 Year Mortgage Rate,Basis
1977-02-18,7.76,8.65,0.89
1977-02-25,7.79,8.65,0.86
1977-03-04,7.82,8.65,0.83
1977-03-11,7.80,8.70,0.90
1977-03-18,7.76,8.70,0.94
1977-03-25,7.78,8.70,0.92
1977-04-01,7.78,8.70,0.92
1977-04-15,7.65,8.75,1.10
1977-04-22,7.73,8.78,1.05
1977-04-29,7.80,8.78,0.98


In [56]:
#Construct chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Bar(name='Originations', x=dfo.index, y=dfo['Qtrly Originations (B)'], text=dfo['Qtrly Originations (B)'],
                     textposition='outside', yaxis='y2'))
fig.update_traces(marker_color='lightblue', opacity=0.5)
fig.add_trace(go.Scatter(x=m.index, y=m['Basis'], mode='lines', name='Mortgage - 30 Year', marker_color='navy'))

fig.update_layout(barmode='stack', title_text='30 Yr Mortage / UST30 Basis - w/Origination Volume', showlegend=False,
                  hoverlabel=dict(bgcolor='white'), template='plotly_white', xaxis=dict(showgrid=False), 
                  yaxis2=dict(title='Large Bank Qtrly Originations (B)',tickformat=',.2f', showgrid=False),
                  yaxis1=dict(title='30 Yr Mortgage - UST30 (bps)', tickformat='.2f', zeroline=False,showgrid=False),
                  layout_xaxis_rangeslider_visible=True)

fig.add_annotation(text='Data: https://fred.stlouisfed.org/',
                  xref='paper', yref='paper',
                  x=0.92, y=-0.15, showarrow=False)

fig.show()

In [60]:
fig.write_html(r'C:\Users\17049\Documents\Charts\30Yr Mortgage UST Basis.html', full_html=False, include_plotlyjs='cdn')